In [1]:
# Import all packages needed
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/theo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.test.utils import datapath

# spacy
import spacy

# gensim logging
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [4]:
df = pd.read_csv('../train_40k.csv')

In [5]:
# convert to list
data = df.Text.values.tolist()

# remove emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# remove single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['The description and photo on this product needs to be changed to indicate '
 'this product is the BuffalOs version of this beef jerky.']


In [6]:
# split senteces to list of words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['the', 'description', 'and', 'photo', 'on', 'this', 'product', 'needs', 'to', 'be', 'changed', 'to', 'indicate', 'this', 'product', 'is', 'the', 'buffalos', 'version', 'of', 'this', 'beef', 'jerky']]


In [7]:
# Build bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# show trigram
print(trigram_mod[bigram_mod[data_words[0]]])

['the', 'description', 'and', 'photo', 'on', 'this', 'product', 'needs', 'to', 'be', 'changed', 'to', 'indicate', 'this', 'product', 'is', 'the', 'buffalos', 'version', 'of', 'this', 'beef_jerky']


In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 
nlp = spacy.load("en_core_web_sm")

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['description', 'photo', 'product', 'need', 'change', 'indicate', 'product', 'version']]


In [10]:
print(data_lemmatized[0])

['description', 'photo', 'product', 'need', 'change', 'indicate', 'product', 'version']


In [11]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1)]]


In [12]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20)

In [13]:
# save the trained model
file_dir = "LDA_Model/LDA.model"
lda_model.save(file_dir)
lda_model = gensim.models.ldamodel.LdaModel.load(file_dir)

In [14]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.053*"cat" + 0.020*"get" + 0.019*"box" + 0.018*"work" + 0.013*"buy" + '
  '0.013*"pet" + 0.012*"go" + 0.010*"litter" + 0.010*"time" + 0.009*"day"'),
 (1,
  '0.026*"price" + 0.024*"use" + 0.021*"good" + 0.020*"brush" + 0.019*"well" + '
  '0.018*"great" + 0.018*"year" + 0.018*"product" + 0.017*"amazon" + '
  '0.015*"work"'),
 (2,
  '0.083*"dog" + 0.018*"work" + 0.015*"product" + 0.014*"get" + 0.013*"pain" + '
  '0.012*"use" + 0.011*"pad" + 0.011*"shampoo" + 0.011*"treat" + 0.011*"try"'),
 (3,
  '0.016*"fit" + 0.013*"easy" + 0.012*"get" + 0.011*"put" + 0.011*"well" + '
  '0.009*"seat" + 0.009*"baby" + 0.009*"also" + 0.009*"little" + 0.008*"side"'),
 (4,
  '0.028*"love" + 0.022*"food" + 0.015*"puppy" + 0.014*"buy" + 0.014*"get" + '
  '0.012*"little" + 0.011*"make" + 0.009*"go" + 0.008*"year" + 0.008*"time"'),
 (5,
  '0.030*"taste" + 0.025*"good" + 0.019*"bottle" + 0.018*"flavor" + '
  '0.016*"water" + 0.015*"try" + 0.014*"eat" + 0.014*"product" + 0.013*"great" '
  '+ 0.013*"well"'